<a href="https://colab.research.google.com/github/edsml-kl121/geeimperial/blob/master/Test_accuracy_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook.  Suppose you want to predict a continuous output (regression) from a stack of continuous inputs.  In this example, the output is impervious surface area from [NLCD](https://www.mrlc.gov/data) and the input is a Landsat 8 composite.  The model is a [fully convolutional neural network (FCNN)](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf), specifically [U-net](https://arxiv.org/abs/1505.04597). This notebook shows:

1.   Exporting training/testing patches from Earth Engine, suitable for training an FCNN model.
2.   Preprocessing.
3.   Training and validating an FCNN model.
4.   Making predictions with the trained model and importing them to Earth Engine.

# Variables

Declare the variables that will be in use throughout the notebook.

In [1]:
PACKAGE_PATH = 'Water_classification_package'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 12
drwxr-xr-x 1 root root 4096 Jul 13 13:43 sample_data
drwxr-xr-x 9 root root 4096 Jul 16 17:51 wandb
drwxr-xr-x 3 root root 4096 Jul 16 09:51 Water_classification_package
mkdir: cannot create directory ‘Water_classification_package’: File exists
total 20
-rw-r--r-- 1 root root 1732 Jul 16 16:59 config.py
-rw-r--r-- 1 root root    0 Jul 16 17:54 __init__.py
-rw-r--r-- 1 root root 6710 Jul 16 17:53 metrics_.py
-rw-r--r-- 1 root root 2844 Jul 16 16:59 preprocessing.py
drwxr-xr-x 2 root root 4096 Jul 16 17:34 __pycache__


In [27]:
%%writefile {PACKAGE_PATH}/metrics_.py

from keras import backend as K
import tqdm.notebook as tq
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

CONFIG = None

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

# Acc = TP + TN / (TP + TN + FP + FN)
# possible_pos = TP + FN
# predicted_pos = TP + FP
# Missing TN
# TN = total - possible_pos - predicted_pos + TP
# TP + TN + FP + FN = possible_pos + predicted_pos - TP + TN

def custom_accuracy(y_true, y_pred):
    # total_data = K.int_shape(y_true) + K.int_shape(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip(1 - y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    total_data = - true_positives + true_negatives + possible_positives + predicted_positives
  
    # total_data = tf.cast(total_data, tf.float32|tf.int32)
    # true_positives = tf.cast(true_positives, tf.float32|tf.int32)
    # possible_positives = tf.cast(possible_positives, tf.float32|tf.int32)
    # predicted_positives = tf.cast(predicted_positives, tf.float32|tf.int32)
    # print(K.int_shape(y_true), K.int_shape(y_pred))
    # print(K.int_shape(y_pred)[0], K.int_shape(y_pred)[1], K.int_shape(y_pred)[2])
    # print(total_data)
    # print(possible_positives)
    # (true_positives) / (total_data + K.epsilon())
    return (true_positives + true_negatives) / (total_data + K.epsilon())



def MetricCalculator_backup(model, test_data, total_steps):
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  # total_steps = 2000
  test_acc_metric = tf.keras.metrics.Accuracy()
  # test_F1_metric = tfa.metrics.F1Score(num_classes=2, threshold=0.5)
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    # print(y_pred[0][1] == 1, y_pred[0][1] == 1)
    for j in range(y_pred.shape[0]):
      if y_true[j][1] == 1 and y_pred[j][1] == 1:
        TP += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 0:
        TN += 1
      if y_true[j][1] == 1 and y_pred[j][1] == 0:
        FN += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 1:
        FP += 1
    test_acc_metric.update_state(y_true, y_pred)
  print("TP: ", TP)
  print("TN: ", TN)
  print("FP: ", FP)
  print("FN: ", FN)


  if TP != 0:
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    F1 = 2*(recall*precision)/(recall + precision)
  else:
    precision = 0
    recall = 0
    F1 = -1

  print("precision: ", precision)
  print("recall: ", recall)
  print("F1_Score: : ", F1)
  print("Accuracy: ", test_acc_metric.result().numpy())
  return precision, recall, F1, test_acc_metric.result().numpy()

def MetricCalculator(model, test_data, total_steps):
  pred = []
  true = []
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)


  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy


def MetricCalculator_multiview_2(model, test_data, total_steps):
  pbar = tq.tqdm(total=total_steps)
  pred = []
  true = []
  for steps, data in enumerate(test_data):
    pbar.update(1)
    if steps >= total_steps:
      break
    input = data[0]
    x1, x2 = tf.split(input, [len(CONFIG.BANDS1),len(CONFIG.BANDS2)], 3)
    y_true = data[1]
    y_pred = np.rint(model.predict([x1, x2]))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)
  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))red, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy

Overwriting Water_classification_package/metrics_.py


In [28]:
%%writefile {PACKAGE_PATH}/config.py
import tensorflow as tf
from . import metrics_
class configuration:
  def __init__(self, PROJECT_TITLE, BANDS1, TRAIN_SIZE, EVAL_SIZE, BANDS2=[], BANDS3=[], country="TH", image=None, sam_arr=None, type_=1):
    if type_ == 1:
      self.type_ = "fs"
    elif type_ == 2:
      self.type_ = "m2"
    elif type_ == 3:
      self.type_ = "m3"
    else:
      self.type_ = None
    self.country = country
    self.PROJECT_TITLE = PROJECT_TITLE
    self.BANDS1 = BANDS1
    self.BANDS2 = BANDS2
    self.BANDS3 = BANDS3
    self.BUCKET = "geebucketwater"
    self.FOLDER = f'{self.type_}_{self.country}_Cnn_{self.PROJECT_TITLE}'
    self.TRAIN_SIZE = TRAIN_SIZE
    self.EVAL_SIZE = EVAL_SIZE
    self.BUCKET = "geebucketwater"
    self.TRAINING_BASE = f'training_patches_{PROJECT_TITLE}'
    self.EVAL_BASE = f'eval_patches_{PROJECT_TITLE}'
    self.TEST_BASE_1 = f'test_patches_{PROJECT_TITLE}_1'
    self.TEST_BASE_2 = f'test_patches_{PROJECT_TITLE}_2'
    self.RESPONSE = 'water'
    self.BANDS = BANDS1 + BANDS2 + BANDS3 
    self.FEATURES = BANDS1 + BANDS2 + BANDS3 + [self.RESPONSE]
    # Specify the size and shape of patches expected by the model.
    self.KERNEL_SIZE = 256
    self.KERNEL_SHAPE = [self.KERNEL_SIZE, self.KERNEL_SIZE]
    self.COLUMNS = [
      tf.io.FixedLenFeature(shape=self.KERNEL_SHAPE, dtype=tf.float32) for k in self.FEATURES
    ]
    self.FEATURES_DICT = dict(zip(self.FEATURES, self.COLUMNS))
    # Specify model training parameters.
    self.BATCH_SIZE = 16
    self.EPOCHS = 5
    self.BUFFER_SIZE = 2000
    self.OPTIMIZER = 'adam'
    self.LOSS = 'categorical_crossentropy'
    self.METRICS = ['AUC', metrics_.f1, metrics_.custom_accuracy]
    self.image = image
    self.sam_arr = sam_arr
    




Overwriting Water_classification_package/config.py


In [9]:
%%writefile {PACKAGE_PATH}/preprocessing.py

import tensorflow as tf

class Preprocessor:
  def __init__(self, config):
    self.config = config

  def parse_tfrecord(self, example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
      example_proto: a serialized Example.
    Returns:
      A dictionary of tensors, keyed by feature name.
    """
    return tf.io.parse_single_example(example_proto, self.config.FEATURES_DICT)


  def to_tuple(self, inputs):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
      inputs: A dictionary of tensors, keyed by feature name.
    Returns:
      A tuple of (inputs, outputs).
    """
    inputsList = [inputs.get(key) for key in self.config.FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(self.config.BANDS)], tf.reshape(tf.one_hot(tf.cast(stacked[:,:,len(self.config.BANDS):], tf.int32), depth=2),[256,256,2])


  def get_dataset(self, pattern):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
      pattern: A file pattern to match in a Cloud Storage bucket.
    Returns:
      A tf.data.Dataset
    """
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(self.parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(self.to_tuple, num_parallel_calls=5)
    return dataset

  def get_training_dataset(self, location):
    """Get the preprocessed training dataset
    Returns: 
      A tf.data.Dataset of training data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "training_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.shuffle(self.config.BUFFER_SIZE).batch(self.config.BATCH_SIZE).repeat()
    return dataset



  def get_eval_dataset(self, location):
    """Get the preprocessed evaluation dataset
    Returns: 
      A tf.data.Dataset of evaluation data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "eval_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset

  # print(iter(evaluation.take(1)).next())

  def get_test_dataset(self, location, test_base):
    """Get the preprocessed evaluation dataset
    Returns: 
      A tf.data.Dataset of evaluation data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + test_base + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset



Overwriting Water_classification_package/preprocessing.py


# Setup software libraries

Authenticate and import as necessary.

In [14]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()


project_id = 'coastal-cell-299117'
!gcloud config set project {project_id}

# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)



To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yc5a-SbJsXMlgL3VJkEO-9cgWc_FsuYqzU5x9w26CE0&tc=No6-vw4JtJ1FeAoqFxvgtfhr7B9TVS12Qlz7ihBQzQQ&cc=jeIXbDVLf8s6fNAwjaiGqsT3r8J_YY51PRiGUzaluYQ

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qicqtlFdLGE9VH5Bul09fG-rUwnpizL9OId3uFNXbuGBHF5qA7qy70

Successfully saved authorization token.
Updated property [core/project].
2.8.2


In [22]:
from importlib import reload
reload(metrics_)
from Water_classification_package import config, metrics_, preprocessing

In [11]:
configs_fs = {}
train_size = 240*3
eval_size = 240*2

#### Feature stack experiment

configs_fs["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle", "elevation", "slope", "aspect"], train_size, eval_size)
configs_fs["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle", "elevation"], train_size, eval_size)
configs_fs["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle", "slope"], train_size, eval_size)
configs_fs["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle", "aspect"], train_size, eval_size)
configs_fs["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle", "slope", "aspect"], train_size, eval_size)
configs_fs["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle", "elevation", "slope"], train_size, eval_size)
configs_fs["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle", "elevation", "aspect"], train_size, eval_size)

configs_fs["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH", "elevation", "slope", "aspect"], train_size, eval_size)
configs_fs["S1_el"] = config.configuration("S1_el", ["VV", "VH", "elevation"], train_size, eval_size)
configs_fs["S1_sl"] = config.configuration("S1_sl", ["VV", "VH", "slope"], train_size, eval_size)
configs_fs["S1_as"] = config.configuration("S1_as", ["VV", "VH", "aspect"], train_size, eval_size)
configs_fs["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH", "slope", "aspect"], train_size, eval_size)
configs_fs["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH", "elevation", "slope"], train_size, eval_size)
configs_fs["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH", "elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope", "aspect"], train_size, eval_size)
configs_fs["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation"], train_size, eval_size)
configs_fs["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "slope"], train_size, eval_size)
configs_fs["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7", "aspect"], train_size, eval_size)
configs_fs["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "slope", "aspect"], train_size, eval_size)
configs_fs["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope"], train_size, eval_size)
configs_fs["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "elevation"], train_size, eval_size)
configs_fs["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope"], train_size, eval_size)
configs_fs["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope","elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "elevation"], train_size, eval_size)
configs_fs["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope"], train_size, eval_size)
configs_fs["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope","elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR"] = config.configuration("L8SR", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size)
configs_fs["S1"] = config.configuration("S1", ["VV", "VH"], train_size, eval_size)
configs_fs["S1A"] = config.configuration("S1A", ["VV", "VH", "angle"], train_size, eval_size)
configs_fs["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH"], train_size, eval_size)
configs_fs["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle"], train_size, eval_size)

###### Multiexperiment

configs_multi = {}

configs_multi["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2)
configs_multi["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle"], train_size, eval_size, ["elevation"], type_=2)
configs_multi["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle"], train_size, eval_size, ["slope"], type_=2)
configs_multi["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle"], train_size, eval_size, ["aspect"], type_=2)
configs_multi["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["slope", "aspect"], type_=2)
configs_multi["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope"], type_=2)
configs_multi["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "aspect"], type_=2)

configs_multi["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2)
configs_multi["S1_el"] = config.configuration("S1_el", ["VV", "VH"], train_size, eval_size, ["elevation"], type_=2)
configs_multi["S1_sl"] = config.configuration("S1_sl", ["VV", "VH"], train_size, eval_size, ["slope"], type_=2)
configs_multi["S1_as"] = config.configuration("S1_as", ["VV", "VH"], train_size, eval_size, ["aspect"], type_=2)
configs_multi["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH"], train_size, eval_size, ["slope", "aspect"], type_=2)
configs_multi["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH"], train_size, eval_size, ["elevation", "slope"], type_=2)
configs_multi["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH"], train_size, eval_size, ["elevation", "aspect"], type_=2)

configs_multi["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2)
configs_multi["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation"], type_=2)
configs_multi["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope"], type_=2)
configs_multi["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["aspect"], type_=2)
configs_multi["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope", "aspect"], type_=2)
configs_multi["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope"], type_=2)
configs_multi["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "aspect"], type_=2)


configs_multi["L8SR_S1_as"] = config.configuration("L8SR_S1_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "aspect"], type_=2)
configs_multi["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation"], type_=2)
configs_multi["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope"], type_=2)

configs_multi["L8SR_S1_sl_as"] = config.configuration("L8SR_S1_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope", "aspect"], type_=2)
configs_multi["L8SR_S1_el_sl"] = config.configuration("L8SR_S1_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "slope"], type_=2)
configs_multi["L8SR_S1_el_as"] = config.configuration("L8SR_S1_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "aspect"], type_=2)
configs_multi["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope","elevation", "aspect"], type_=2)

configs_multi["L8SR_S1A_as"] = config.configuration("L8SR_S1A_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "aspect"], type_=2)
configs_multi["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation"], type_=2)
configs_multi["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope"], type_=2)

configs_multi["L8SR_S1A_sl_as"] = config.configuration("L8SR_S1A_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope", "aspect"], type_=2)
configs_multi["L8SR_S1A_el_sl"] = config.configuration("L8SR_S1A_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "slope"], type_=2)
configs_multi["L8SR_S1A_el_as"] = config.configuration("L8SR_S1A_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "aspect"], type_=2)
configs_multi["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope","elevation", "aspect"], type_=2)

configs_multi["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], type_=2)
configs_multi["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], type_=2)

# configs_multi["L8SR_S1_as3"] = config.configuration("L8SR_S1_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["aspect"], type_=3)
# configs_multi["L8SR_S1_el3"] = config.configuration("L8SR_S1_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["elevation"], type_=3)
# configs_multi["L8SR_S1_sl3"] = config.configuration("L8SR_S1_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope"], type_=3)
# configs_multi["L8SR_S1_sl_el_as3"] = config.configuration("L8SR_S1_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope","elevation", "aspect"], type_=3)

# configs_multi["L8SR_S1A_as3"] = config.configuration("L8SR_S1A_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["aspect"], type_=3)
# configs_multi["L8SR_S1A_el3"] = config.configuration("L8SR_S1A_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["elevation"], type_=3)
# configs_multi["L8SR_S1A_sl3"] = config.configuration("L8SR_S1A_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope"], type_=3)
# configs_multi["L8SR_S1A_sl_el_as3"] = config.configuration("L8SR_S1A_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope","elevation", "aspect"], type_=3)

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [7]:
%%capture
!pip install wandb --upgrade

In [8]:
import wandb
from wandb.keras import WandbCallback
import time
import pandas as pd

# wandb.init(reinit=True)

wandb.init(project='kl-121-dissertation', reinit=True)

df_test1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_m2_1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_m2_2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])

wandb: Currently logged in as: kl-121 (kl-121-dissertation). Use `wandb login --relogin` to force relogin


# FeatureStack

In [23]:
for i in range(0, len(list(configs_fs))):
  conf = configs_fs[list(configs_fs)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  wandb.run.name = "test_accuracy_feature_stack_tibet_thailand"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})
  test_1 = preproc.get_test_dataset("Train_in_Thailand/", "test_patches_1")
  test_2 = preproc.get_test_dataset("Train_in_Thailand/", "test_patches_1")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator(model_custom, test_1, 240)
  precision_t2, recall_t2, F1_t2, accuracy_t2 = metrics_.MetricCalculator(model_custom, test_2, 240)
  df_test1.loc[i] = [conf.PROJECT_TITLE + " test1"] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  df_test2.loc[i] = [conf.PROJECT_TITLE + " test2"] + \
  [precision_t2] + [recall_t2] + [F1_t2] + [accuracy_t2]
  wandb.log({'df_featurestack_test1_': df_test1})
  wandb.log({'df_featurestack_test2_': df_test2})

  0%|          | 0/240 [00:00<?, ?it/s]

ValueError: ignored

# Multiview-2

In [13]:
for i in range(0, len(list(configs_multi))):
  print(i)
  conf = configs_multi[list(configs_multi)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  wandb.run.name = "test_accuracy_multiview-2_tibet_thailand"
  metrics_.CONFIG = conf
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})
  test_1 = preproc.get_test_dataset("Train_in_Thailand/", "test_patches_1")
  test_2 = preproc.get_test_dataset("Train_in_Thailand/", "test_patches_2")
  precision_t1, recall_t1, F1_mac_t1, accuracy_t1 = metrics_.MetricCalculator_multiview_2(model_custom, test_1, 240)
  precision_t2, recall_t2, F1_mac_t2, accuracy_t2 = metrics_.MetricCalculator_multiview_2(model_custom, test_2, 240)
  df_test_m2_1.loc[i] = [conf.PROJECT_TITLE + " test1"] + \
  [precision_t1] + [recall_t1] + [F1_mac_t1] + [accuracy_t1]
  df_test_m2_2.loc[i] = [conf.PROJECT_TITLE + " test2"] + \
  [precision_t2] + [recall_t2] + [F1_mac_t2] + [accuracy_t2]
  wandb.log({'df_multiview-2_test1_': df_test_m2_1})
  wandb.log({'df_multiview-2_test2_': df_test_m2_2})

0
gs://geebucketwater/m2_TH_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.9797519759533122
recall_macro:  0.9720049867060336
F1_macro_Score: :  0.9758364025617808
F1_micro_Score: :  0.990457026163737
Accuracy:  0.0


  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.8403483824944535
recall_macro:  0.500085639717635
F1_macro_Score: :  0.4409260787606839
F1_micro_Score: :  0.7881141662597657
Accuracy:  0.0
1
gs://geebucketwater/m2_TH_Cnn_S1A_el/Models/S1A_el_EPOCHS_10
S1A_el


...

precision_macro:  0.9854083727879308
recall_macro:  0.9575183196954669
F1_macro_Score: :  0.9709169682424451
F1_micro_Score: :  0.9887519200642905
Accuracy:  0.0


  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.6747864263276967
recall_macro:  0.6599858864837973
F1_macro_Score: :  0.6664797399356858
F1_micro_Score: :  0.7864439260623907
Accuracy:  0.0
3
gs://geebucketwater/m2_TH_Cnn_S1A_as/Models/S1A_as_EPOCHS_10
S1A_as


Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 170, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 127, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 395, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 226, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/wa

KeyboardInterrupt: ignored

In [18]:

conf = configs_fs[list(configs_fs)[0]]
preproc = preprocessing.Preprocessor(conf)
MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})
test_1 = preproc.get_test_dataset("Train_in_Thailand/", "test_patches_1")

In [ ]:
test_1

In [19]:
testdat = model_custom.evaluate(test_1, steps=240)

240/240 [==============================] - 52s 210ms/step - loss: 0.0344 - auc: 0.9988 - f1: 0.9874 - custom_accuracy: 0.9879


In [98]:
testdat.shape

(240, 256, 256, 2)

In [111]:
# np.logical_and(testdat < 0.5, testdat > 0.51).all()
np.logical_and(testdat < 0.45, testdat > 0.5).all()
# (testdat > 0.4 and testdat < 0.6)

False

In [109]:
(testdat <1.01).all()

True

# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storge bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.

The following functions handle this process.  It's useful to separate the export from the predictions so that you can experiment with different models without running the export every time.

In [ ]:
def doExport(out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(config.BANDS),
    description = out_image_base,
    bucket = config.BUCKET,
    fileNamePrefix = config.FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': config.KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
import json
from pprint import pprint
import numpy as np
import tqdm.notebook as tq

def LoadImage(out_image_base, user_folder, kernel_buffer, region):
  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{config.BUCKET}'/'{config.FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  pprint(imageFilesList)
  print(jsonFile)

  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      config.KERNEL_SHAPE[0] + kernel_buffer[0],
      config.KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in config.BANDS
  ]

  imageFeaturesDict = dict(zip(config.BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in config.BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
  return imageDataset, patches, x_buffer, y_buffer, jsonFile

def predictionSingleinput(model, imageDataset, patches):
  print('Running predictions...')
  predictions = model.predict(imageDataset, steps=patches, verbose=1)
  return predictions

def predictionMultipleinput(model, imageDataset, patches):
  print('Running predictions...')
  predictions = []
  pbar = tq.tqdm(total=patches)
  for data in imageDataset:
    pbar.update(1)
    x1, x2 = tf.split(data, [6,3], 3)
    predictions.append(model_custom_1EP.predict([x1, x2], verbose=0))
  return predictions

def uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, multiview=False):
  print('Writing predictions...')
  out_image_file = 'gs://' + config.BUCKET + '/' + config.FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    if multiview == True:
      predictionPatch = predictionPatch[0]
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+config.KERNEL_SIZE, y_buffer:y_buffer+config.KERNEL_SIZE]
    predictionPatch = np.argmax(predictionPatch, -1)
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base + suffix
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region, model, suffix):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  imageDataset, patches, x_buffer, y_buffer, jsonFile = LoadImage(out_image_base, user_folder, kernel_buffer, region)
  predictions = predictionSingleinput(model, imageDataset, patches)
  uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, False)
  return

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/mewchayutaphong' # INSERT YOUR FOLDER HERE.

# Half this will extend on the sides of each patch.
kernel_buffer = [128, 128]
bj_kernel_buffer = [128, 128]
# Base file name to use for TFRecord files and assets.
bj_image_base = 'FCNN_demo_beijing_384_S1'
# Beijing
bj_region = ee.Geometry.Polygon(
        [[[115.9662455210937, 40.121362012835235],
          [115.9662455210937, 39.64293313749715],
          [117.01818643906245, 39.64293313749715],
          [117.01818643906245, 40.121362012835235]]], None, False)

th_image_base = f'Thai_water_with_{config.PROJECT_TITLE}'
th_region = ee.Geometry.BBox(106.42682423644103, 15.001360239504, 106.8014457430141, 15.59647593973863)


np_image_base = f'Nepal_water_with_{config.PROJECT_TITLE}'
np_region = ee.Geometry.BBox(70.3090276140532, 31.351437257990685, 71.5724553484282, 32.32209892418571)


In [ ]:
# Run the export.
# doExport(bj_image_base, bj_kernel_buffer, bj_region)
doExport(np_image_base, kernel_buffer, np_region)

Running image export to Cloud Storage...
Image export completed.


In [ ]:
doExport(th_image_base, kernel_buffer, th_region)

Running image export to Cloud Storage...
Image export completed.


In [ ]:
doPrediction(th_image_base, user_folder, kernel_buffer, th_region, model_custom_1EP, "_epochs_1_")
doPrediction(th_image_base, user_folder, kernel_buffer, th_region, model_custom_2EP, "_epochs_2_")
doPrediction(th_image_base, user_folder, kernel_buffer, th_region, model_custom_3EP, "_epochs_3_")

Looking for TFRecord files...
['gs://geebucketwater/Cnn_S1_SLOPE/Thai_water_with_S1_SLOPE.tfrecord.gz']
gs://geebucketwater/Cnn_S1_SLOPE/Thai_water_with_S1_SLOPE.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 5,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            106.42664564466256,
                                            0.0,
                                            -0.00026949458523585647,
                                            15.596729625939957]},
                'crs': 'EPSG:4326'},
 'totalPatches': 40}
Running predictions...
40/40 [==============================] - 11s 163ms/step
Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Writing patch 12...
Writi

In [ ]:
# Run the prediction.
# doPrediction(bj_image_base, user_folder, bj_kernel_buffer, bj_region)
doPrediction(np_image_base, user_folder, kernel_buffer, np_region, model_custom_1EP, "_epochs_1")
doPrediction(np_image_base, user_folder, kernel_buffer, np_region, model_custom_2EP, "_epochs_2")
doPrediction(np_image_base, user_folder, kernel_buffer, np_region, model_custom_3EP, "_epochs_3")

Looking for TFRecord files...
['gs://geebucketwater/Cnn_S1_SLOPE/Nepal_water_with_S1_SLOPE00000.tfrecord.gz',
 'gs://geebucketwater/Cnn_S1_SLOPE/Nepal_water_with_S1_SLOPE00001.tfrecord.gz',
 'gs://geebucketwater/Cnn_S1_SLOPE/Nepal_water_with_S1_SLOPE00002.tfrecord.gz',
 'gs://geebucketwater/Cnn_S1_SLOPE/Nepal_water_with_S1_SLOPE00003.tfrecord.gz',
 'gs://geebucketwater/Cnn_S1_SLOPE/Nepal_water_with_S1_SLOPE00004.tfrecord.gz',
 'gs://geebucketwater/Cnn_S1_SLOPE/Nepal_water_with_S1_SLOPE00005.tfrecord.gz']
gs://geebucketwater/Cnn_S1_SLOPE/Nepal_water_with_S1_SLOPEmixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 18,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            70.30898133135307,
                                            0.0,
                                            -0.00026949458523585647,
                                            32.3237195423591]},
     

## Accuracy

In [ ]:
# # from tqdm import tqdm
# import tqdm.notebook as tq
# import numpy as np

# def MetricCalculator(model, test_data, total_steps):
#   TP = 0
#   TN = 0
#   FP = 0
#   FN = 0
#   # total_steps = 2000
#   test_acc_metric = tf.keras.metrics.Accuracy()
#   # test_F1_metric = tfa.metrics.F1Score(num_classes=2, threshold=0.5)
#   pbar = tq.tqdm(total=total_steps)
#   for steps, data in enumerate(test_data):
#     # print(f'Number of steps: {steps}', end = "\r")
#     pbar.update(1)
#     if steps == total_steps:
#       break
#     input = data[0]
#     y_true = data[1]
#     y_pred = np.rint(model.predict(input))
#     y_true = np.reshape(y_true, (256*256,2))
#     y_pred = np.reshape(y_pred, (256*256,2))
#     # print(y_pred[0][1] == 1, y_pred[0][1] == 1)
#     for j in range(y_pred.shape[0]):
#       if y_true[j][1] == 1 and y_pred[j][1] == 1:
#         TP += 1
#       if y_true[j][1] == 0 and y_pred[j][1] == 0:
#         TN += 1
#       if y_true[j][1] == 1 and y_pred[j][1] == 0:
#         FN += 1
#       if y_true[j][1] == 0 and y_pred[j][1] == 1:
#         FP += 1
#     test_acc_metric.update_state(y_true, y_pred)
#     # # recall_metric.update_state(data[1], test_logits)
#     # # precision_metric.update_state(data[1], test_logits)
#   print("TP: ", TP)
#   print("TN: ", TN)
#   print("FP: ", FP)
#   print("FN: ", FN)

#   precision = TP/(TP + FP)
#   recall = TP/(TP + FN)
#   F1 = 2*(recall*precision)/(recall + precision)

#   print("precision: ", precision)
#   print("recall: ", recall)
#   print("F1_Score: : ", F1)
#   print("Accuracy: ", test_acc_metric.result().numpy())
#   return precision, recall, F1, test_acc_metric.result().numpy()

In [ ]:
metrics_.MetricCalculator(model_custom_1EP, test_1, 2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

TP:  1212873
TN:  129047990
FP:  54446
FN:  756691
precision:  0.9570384409923626
recall:  0.6158078640755009
F1_Score: :  0.7494079952843521
Accuracy:  0.9938122


(0.7494079952843521, 0.9938122)

In [ ]:
metrics_.MetricCalculator(model_custom_2EP, test_1, 2000)

In [ ]:
metrics_.MetricCalculator(model_custom_3EP, test_1, 2000)

In [ ]:
metrics_.MetricCalculator(model_custom_1EP, test_2, 2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
metrics_.MetricCalculator(model_custom_2EP, test_2, 2000)

In [ ]:
metrics_.MetricCalculator(model_custom_3EP, test_2, 2000)

# Display the output

One the data has been exported, the model has made predictions and the predictions have been written to a file, and the image imported to Earth Engine, it's possible to display the resultant Earth Engine asset.  Here, display the impervious area predictions over Beijing, China.

In [ ]:
user_folder + '/' + bj_image_base

In [ ]:
out_image = ee.Image(user_folder + '/' + bj_image_base)
mapid = out_image.getMapId({'min': 0, 'max': 2})
map = folium.Map(location=[39.898, 116.5097])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map